In [2]:
import pandas as pd
import os
import librosa as lr
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow import keras

In [3]:
from glob import glob
import numpy as np

In [4]:
data_dir = "./Data/genres_original/"
sr = 22050

In [5]:
df = pd.read_csv("./Data/features_extraction.csv")
df

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,rock.00095.wav,661794,0.352063,0.080487,0.079486,0.000345,2008.149458,282174.689224,2106.541053,88609.749506,...,45.050526,-13.289984,41.754955,2.484145,36.778877,-6.713265,54.866825,-1.193787,49.950665,rock
995,rock.00096.wav,661794,0.398687,0.075086,0.076458,0.000588,2006.843354,182114.709510,2068.942009,82426.016726,...,33.851742,-10.848309,39.395096,1.881229,32.010040,-7.461491,39.196327,-2.795338,31.773624,rock
996,rock.00097.wav,661794,0.432142,0.075268,0.081651,0.000322,2077.526598,231657.968040,1927.293153,74717.124394,...,33.597008,-12.845291,36.367264,3.440978,36.001110,-12.588070,42.502201,-2.106337,29.865515,rock
997,rock.00098.wav,661794,0.362485,0.091506,0.083860,0.001211,1398.699344,240318.731073,1818.450280,109090.207161,...,46.324894,-4.416050,43.583942,1.556207,34.331261,-5.041897,47.227180,-3.590644,41.299088,rock


In [6]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """

    h = array.shape[0]
    w = array.shape[1]
    
    a = (xx - h) // 2
    aa = xx - a - h

    b = (yy - w) // 2
    bb = yy - b - w

    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [7]:
def get_features(df_in):
    features=[] 
    labels=[] 
    for index in range(0,len(df_in)):     
      filename = df_in.iloc[index]['filename']  
      tstart = 0 
      tend = df_in.iloc[index]['length'] 
      species_id = df_in.iloc[index]['label']         
      y, sr = librosa.load('Data/genres_original/'+filename,sr=22050)
      y_cut = y[round(tstart*sr,ndigits=None)
         :round(tend*sr, ndigits= None)]
      data = np.array([padding(librosa.feature.mfcc(y_cut, 
         n_fft=255,hop_length=512,n_mfcc=128),130,1500)])
      features.append(data)
      labels.append(species_id)
    output=np.concatenate(features,axis=0)
    return(np.array(output), labels)
X,y=get_features(df)

In [8]:
X = np.array((X-np.min(X))/(np.max(X)-np.min(X)))
X = X/np.std(X)
y = np.array(y)

In [9]:
from sklearn.preprocessing import LabelEncoder
names_list = y
con = LabelEncoder()
y = con.fit_transform(names_list)


In [10]:
#Split twice to get the validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123)
#Print the shapes
X_train.shape, X_test.shape, X_val.shape, len(y_train), len(y_test), len(y_val)

((561, 130, 1500), (250, 130, 1500), (188, 130, 1500), 561, 250, 188)

In [11]:
input_shape=(130,1500)
model = tensorflow.keras.Sequential()
model.add(LSTM(128,input_shape=input_shape))
model.add(Dense(24, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               834048    
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
Total params: 837,144
Trainable params: 837,144
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',loss='SparseCategoricalCrossentropy',metrics=['acc'])

In [13]:
history = model.fit(X_train, y_train, epochs=50, batch_size=72, 
                    validation_data=(X_val, y_val), shuffle=False)

Epoch 1/50
8/8 [==============================] - ETA: 23s - loss: 3.5331 - acc: 0.0000e+ - ETA: 4s - loss: 3.3263 - acc: 0.0486     - ETA: 3s - loss: 3.2023 - acc: 0.064 - ETA: 2s - loss: 3.1676 - acc: 0.059 - ETA: 2s - loss: 3.1235 - acc: 0.063 - ETA: 1s - loss: 3.0990 - acc: 0.074 - ETA: 0s - loss: 3.0610 - acc: 0.085 - ETA: 0s - loss: 3.0357 - acc: 0.087 - 10s 926ms/step - loss: 3.0357 - acc: 0.0873 - val_loss: 2.7684 - val_acc: 0.1170
Epoch 2/50
8/8 [==============================] - ETA: 4s - loss: 2.7426 - acc: 0.083 - ETA: 4s - loss: 2.7497 - acc: 0.090 - ETA: 3s - loss: 2.7267 - acc: 0.106 - ETA: 2s - loss: 2.7358 - acc: 0.114 - ETA: 2s - loss: 2.7248 - acc: 0.116 - ETA: 1s - loss: 2.7218 - acc: 0.120 - ETA: 0s - loss: 2.7108 - acc: 0.113 - ETA: 0s - loss: 2.7016 - acc: 0.112 - 7s 864ms/step - loss: 2.7016 - acc: 0.1123 - val_loss: 2.6183 - val_acc: 0.0851
Epoch 3/50
8/8 [==============================] - ETA: 4s - loss: 2.5917 - acc: 0.166 - ETA: 4s - loss: 2.5960 - acc: 0.14

Epoch 20/50
8/8 [==============================] - ETA: 4s - loss: 2.3429 - acc: 0.027 - ETA: 4s - loss: 2.3303 - acc: 0.076 - ETA: 4s - loss: 2.3244 - acc: 0.097 - ETA: 3s - loss: 2.3201 - acc: 0.107 - ETA: 2s - loss: 2.3216 - acc: 0.102 - ETA: 1s - loss: 2.3188 - acc: 0.108 - ETA: 0s - loss: 2.3200 - acc: 0.103 - ETA: 0s - loss: 2.3187 - acc: 0.101 - 7s 933ms/step - loss: 2.3187 - acc: 0.1016 - val_loss: 2.3342 - val_acc: 0.0798
Epoch 21/50
8/8 [==============================] - ETA: 5s - loss: 2.3417 - acc: 0.027 - ETA: 4s - loss: 2.3292 - acc: 0.076 - ETA: 3s - loss: 2.3233 - acc: 0.097 - ETA: 2s - loss: 2.3191 - acc: 0.107 - ETA: 2s - loss: 2.3205 - acc: 0.102 - ETA: 1s - loss: 2.3177 - acc: 0.108 - ETA: 0s - loss: 2.3189 - acc: 0.103 - ETA: 0s - loss: 2.3177 - acc: 0.101 - 7s 818ms/step - loss: 2.3177 - acc: 0.1016 - val_loss: 2.3332 - val_acc: 0.0798
Epoch 22/50
8/8 [==============================] - ETA: 4s - loss: 2.3407 - acc: 0.027 - ETA: 5s - loss: 2.3282 - acc: 0.076 - ETA

KeyboardInterrupt: 

In [ ]:
x = np.array(df.iloc[1,1:-1])
x = np.asarray(x).astype('float')
x = x[np.newaxis, :]
x

In [ ]:
model.predict(df.)

In [299]:
mfccs = []
for name in df.iloc[:,0]:
    audio_file = glob(data_dir + name)
    audio,sfreq = lr.load(audio_file[0],sr=sr)
    mfccs.append(addmfcc(audio))
    

In [133]:
new_mfccs = {}
for i in range(20):
    new_mfccs['mfcc'+str(i+1)] = [mfccs[j][i] for j in range(len(mfccs))]
